As pessoas geralmente dizer "escreva códigos funcionais", isso significa, no contexto de funções que:

1. **Atomicidade**: Uma função deve executar apenas uma atividade

2. **Idempotência**: Se você rodar o código multiplas vezes com a mesma entrada, a saída também deve ser a mesma.
    a. Caso se armazene a saída em um local externo, a saída não pode ser armazenada de forma estar duplicada

3. **Sem efeitos colaterais**: Uma função não deve afetar nenhum dado externo (variável ou outro) além de sua saída.


Observa-se o exemplo abaixo de uma script ETL.

In [ ]:
import praw
import os
import sqlite3
import logging
from dataclasses import asdict

In [2]:
REDDIT_CLIENT_ID=os.environ["REDDIT_CLIENT_ID"]
REDDIT_CLIENT_SECRET=os.environ["REDDIT_CLIENT_SECRET"]
REDDIT_USER_AGENT=os.environ["REDDIT_USER_AGENT"]

In [3]:
def extract():
    client = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secrect=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT,
    )

    subreddit = client.subreddit('dataengineering')
    top_subreddit = subreddit.hot(limit=100)
    data = []

    for submission in top_subreddit:
        data.append(
            {
                'title': submission.title,
                'score': submission.score,
                'id': submission.id,
                'url': submission.url, 
                'comments': submission.num_comments,
                'created': submission.created,
                'text': submission.selftext,
            }
        )
    return data

In [4]:
def transform(data):
    """
    Function to only keep outliers.
    Outliers are based on num of comments > 2 standard deviations from mean
    """
    num_comments = [post.get('comments') for post in data]
    mean_num_comments = sum(num_comments) / len(num_comments)

    std_num_comments = (sum([(x - mean_num_comments) ** 2 for x in num_comments]) / len(num_comments)) ** 0.5

    return [
        post
        for post in data
        if post.get('comments') > mean_num_comments + 2 * std_num_comments
    ]

In [6]:
def load(data):
    #Create a database connection
    conn = sqlite3.connect('./data/socialetl.db')
    cur = conn.cursor()
    try:
        #Insert data into database
        for post in data:
            cur.execute(
                """
                    INSERT INTO social_posts (
                        id, source, social_data
                    ) VALUES (
                        :id, :source, :social_data
                    )
                    """,
                {
                    'id': post.get('id'),
                    'score': post.get('score'),
                    'social_data': str(
                        {
                            'title': post.get('title'),
                            'url': post.get('url'),
                            'comments': post.get('num_comments'),
                            'created': post.get('created'),
                            'text': post.get('selftext')
                        }
                    )
                }
            )
    finally:
        conn.commit()
        conn.close()

Ao se examinar a função 'Load' acima, percebe-se que:

1. **Atomicidade**: Não possui essa característica pois realiza duas atividades: gerencia a conexão com a banco de dados e carrega os dados na banco de dados.
    a. Uma alternativa para se contornar essa situação é a técnica de Injeção de Dependência para aceitar a conexão com a banco de dados como entrada da função 'Load'

2. **Idempotência**: Não possui essa característica pois insere todoso os dados na tabela *social_posts*. Dessa forma, caso a entrada possua duplicatas ou a função seja executada, acidentalmente, duas vezes, valores duplicados serão inseridos na tabela.
    a. É possível prever isso utilizando um UPSERT, que irá atualizar ou inserir uma gravação na banco de dados dependendo se já se está presente ou não.

3. **Sem efeitos colaterais**: A função 'Load' não possui efeitos colaterais. 
    a. Caso a conexão com o banco de dados seja aceito como um parâmetro de entrada da função (Injeção de Dependência), nós não devemos fechat/encerrar essa conexão, visto que isso irá afetar o estado de uma variável externa à função 'Load'.

Agora vamos refatorar a função para corrigir alguns desses problemas

In [ ]:
def load(social_data, db_conn) -> None:
    logging.info('Loading twitter data.')
    if db_conn is None:
        raise ValueError(
            'db_cursos is None. Please pass a valid DatabaseConnection'
            'object.'
        )
    cur = db_conn.cursor()
    try:
        for post in social_data:
            cur.execute(
                """
                INSERT OR REPLACE INTO social_posts (
                    id, source, social_data
                ) VALUES (
                    :id, :source, :social_data
                )
                """,
                {
                    'id': post.id,
                    'source': post.source,
                    'social_data': str(asdict(post.social_data)),
                },
            )
    finally:
        cur.close()
        db_conn.commit()

*Observação*: O cursor deveria ser criado fora da função 'Load', mas não vamos focar nisso agora.

In [7]:
def main():
    #Pull data from reddit
    data = extract()
    #Transform reddit data
    transformed_data = transform(data)
    #Load data into database
    load(transformed_data)